In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('klasik_processed/**/*/*.json', recursive = True)

len(files)

1089

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [4]:
threshold = -18

def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = threshold, minimum_length = 2.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def detect_extra(word):
    word = clean(word)
    return word in [
        'eh', 'ehh', 'oh', 'ohh', 'uhm', 'uhmm',
        'um', 'em', 'emm', 'ah', 'ha', 'ok', 'okay',
        'so', 'yes', 'no', 'ah', 'aa', 'so', 'uh', 'um', 'eh', 'ha', 'oh', 'ye', 'haa',
        'oi', 'ya', 'leh', 'lah', 'haiya', 'hoi', 'haha', 'hahaha',
        'then', 'it s'
    ]

from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [5]:
!rm -rf classic-malay-chunk
!mkdir classic-malay-chunk

In [6]:
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            for a in align:
                if detect_extra(a['text']):
                    a['score'] = 0.0
                
            scores = [a for a in align if a['score'] <= threshold]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    if len(c) == len(align):
                        continue
                    try:
                        
                        words = [c_['text'] for c_ in c if len(c_['text']) <= 1]
                        if (len(words) / len(c)) > 0.5:
                            print(c)
                            continue
                        
                        skip = False
                        
                        for c_ in c:
                            if (c_['end'] - c_['start']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                        
                        for i in range(1, len(c), 1):
                            if (c[i]['start'] - c[i - 1]['end']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                            
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('classic-malay-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [7]:
d = loop((files[-100:], 0))

100%|███████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  5.89it/s]


In [8]:
len(d)

302

In [9]:
d[-1]

{'audio': 'classic-malay-chunk/klasik_processed_KOTA KUALA KEDAH ~ BENTENG PERTAHANAN MELAYU TERTUA [nfQR9iw6RhE]_KOTA KUALA KEDAH ~ BENTENG PERTAHANAN MELAYU TERTUA [nfQR9iw6RhE]_16_0.mp3',
 'transcription': 'Namun, kota ini juga tidak dapat lari daripada kesan perang apabila musnah diserang musuh sebelum ia dibina semula pada tahun 1711 dan diteruskan kerja-kerja membaiki dan menaik taraf semasa era Sultan'}

In [10]:
import IPython.display as ipd
ipd.Audio(d[-1]['audio'])

In [11]:
data = mp.multiprocessing(files, loop, cores = 15)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:25<00:00,  2.84it/s]


In [12]:
len(data)

2814

In [13]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|████████████████████████████████████████████████████████████████████████████████| 2814/2814 [00:00<00:00, 1717516.22it/s]


defaultdict(int, {'classic-malay-chunk/klasik': 2814})

In [14]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

audio  \
0  classic-malay-chunk/klasik_processed_Azzam Sha...   
1  classic-malay-chunk/klasik_processed_Azzam Sha...   
2  classic-malay-chunk/klasik_processed_Azzam Sha...   
3  classic-malay-chunk/klasik_processed_Azzam Sha...   
4  classic-malay-chunk/klasik_processed_Azzam Sha...   

                                       transcription  
0  dan salam satu Malaysia. Okay, saya Azam Syam....  
1  saya yang nak daftar jadi ahli klub Azam Big F...  
2  Haa, ini dia. Anda boleh, aa, beli, aa, produk...  
3      ni. Aa, sebab diorang pun ada buat promotion.  
4  Nak letak saya sayang awak ke? Bagian-bagian u...

In [15]:
df.to_parquet('verify-text-chunk-classic-malay.parquet')

In [16]:
!du -hs classic-malay-chunk

107M	classic-malay-chunk


In [17]:
!zip -rq classic-malay-chunk.zip classic-malay-chunk

In [18]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-classic-malay.parquet",
    path_in_repo="data/classic_malay_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/660b6e1fb0adb90ef05ae511eee284e621d1761c', commit_message='Upload data/classic_malay_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='660b6e1fb0adb90ef05ae511eee284e621d1761c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)

In [19]:
api.upload_file(
    path_or_fileobj="classic-malay-chunk.zip",
    path_in_repo="data/classic-malay-chunk.zip",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)

classic-malay-chunk.zip:   0%|          | 0.00/104M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/28148c2d25269fea6143b63ae82d99e7f6c2863e', commit_message='Upload data/classic-malay-chunk.zip with huggingface_hub', commit_description='', oid='28148c2d25269fea6143b63ae82d99e7f6c2863e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)

In [20]:
!zip -rq malaysian-podcast_processed_alignment.zip malaysian-podcast_processed_alignment

In [21]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="malaysian-podcast_processed_alignment.zip",
    path_in_repo="malaysian-podcast_processed_alignment.zip",
    repo_id="mesolitica/Malaysian-Emilia-annotated",
    repo_type="dataset",
)

malaysian-podcast_processed_alignment.zip:   0%|          | 0.00/266M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-annotated/commit/619a02134493522f2c17e7ec90c2545da9bae8e4', commit_message='Upload malaysian-podcast_processed_alignment.zip with huggingface_hub', commit_description='', oid='619a02134493522f2c17e7ec90c2545da9bae8e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-annotated', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Emilia-annotated'), pr_revision=None, pr_num=None)